# Building GPT From Scratch

## 1. Data Loading and Preparation

### Reading the Text Data

In [12]:
import torch

In [1]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

### Inspecting the Data Size

In [2]:
print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


### Previewing the Dataset

In [5]:
print(text[:100])# print the first 100 characters

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


## 2. Tokenization

### Creating the Vocabulary
A neural network can't process raw characters hence it needs numbers. Therefore our first step is to create a "vocabulary" of all the unique characters present in our text. We also calculate the `vocab_size` which is the number of unique characters. This will determine the size of our model's embedding and output layers.

In [7]:
chars = sorted(list(set(text)))# create a sorted list of unique characters
vocab_size = len(chars)
print("All the unique characters in the dataset are as follows: ", ''.join(chars))

All the unique characters in the dataset are as follows:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
print("Vocabulary size:", vocab_size)

Vocabulary size: 65


### Creating Encoder and Decoder

Now we create a mapping (or "tokenizer") that can convert characters to integers and back.
- **`stoi` (string-to-integer):** A dictionary that maps each unique character to a unique integer.
- **`itos` (integer-to-string):** A dictionary that does the reverse, mapping integers back to characters.
- `encode`: A function that takes a string and converts it into a list of integers (tokens) using stoi.
- `decode`: A function that takes a list of integers and converts it back into a string using itos.

In [10]:
# create a mapping from characters to integers and vice versa
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [11]:
print(encode("hello world")) # encode the string "hello world" to integers
print(decode(encode("hello world"))) # decode back to string

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


### Encoding the Entire Dataset

In [13]:
data = torch.tensor(encode(text), dtype=torch.long)# encode the entire text dataset and store it in a torch tensor
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])
